In [12]:
#%pip install liwc

In [1]:
import nltk
import string
import re
from collections import Counter
import pandas as pd
import spacy
from itertools import compress
import liwc
parse, category_names = liwc.load_token_parser('data/dict_r.dic')

In [2]:
#!python -m spacy download es_core_news_sm
#nltk.download('punkt')
#nltk.download('stopwords')

In [3]:
nlp = spacy.load('es_core_news_sm')

In [4]:
def preprocess(institutions, concept_name):
  institutions = institutions[institutions["categoria"] == concept_name]
  institutions['fundamento'] = institutions['fundamento'].str.lower()
  institutions['fundamento'] = institutions['fundamento'].apply(lambda x: re.sub("[0-9]", '', x))
  institutions['fundamento'] = institutions['fundamento'].str.translate(str.maketrans(' ', ' ', string.punctuation))
  institutions['fundamento'] = institutions['fundamento'].str.strip()
  institutions['fundamento'] = institutions['fundamento'].str.replace('\n', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('...', '')
  institutions['fundamento'] = institutions['fundamento'].str.replace('  ', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('   ', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('    ', ' ')
  institutions['fundamento'] = institutions['fundamento'].str.replace('     ', ' ')

  institutions = institutions[[len(x.split()) > 2 for x in institutions.fundamento]]
  institutions = institutions.reset_index(drop=True)

  return institutions

In [5]:
institutions = pd.read_excel('data/ELA - Instituciones.xlsx')

In [6]:
nlp = spacy.load('es_core_news_sm')

In [12]:
%%time

df = pd.DataFrame()
l_derechos = []

for ix in range(institutions.shape[0]):
  texto = str(institutions.fundamento[ix]).lower()
  doc = nlp(texto)
  tokens = [word.orth_ for word in doc]
  tcounts1 = Counter(category for token in tokens for category in parse(token))
  palabras_derechos = list(compress(tokens, ['derechos' in parse(token) for token in tokens]))
  df_counts = pd.DataFrame.from_dict([tcounts1]).reset_index()
  df_counts['index'] = ix
  df_counts['texto'] = texto
  df_counts['word_derechos'] =  ['' if len(palabras_derechos)==0 else palabras_derechos]
  df = pd.concat([df,df_counts], axis=0, ignore_index=True)

  if len(palabras_derechos)>0:
        l_derechos.extend(palabras_derechos)

CPU times: user 6min 30s, sys: 2.3 s, total: 6min 32s
Wall time: 6min 33s


In [11]:
df_MFD_invir = pd.DataFrame.from_dict(Counter(l_derechos), orient = 'index').reset_index()
df_MFD_invir = df_MFD_invir.rename(columns = {'index':'term', 0:'count'})
df_MFD_invir.sort_values('count', ascending = False)[0:20]

,term,count
13,defender,398
1,tomar,367
6,ejercer,321
0,cumplir,320
10,proteger,311
5,respetar,165
7,conocer,141
3,decir,108
4,trabajar,104
2,responder,85
